In [1]:
!git clone https://github.com/akash092/AI_Healthcare.git

Cloning into 'AI_Healthcare'...
remote: Enumerating objects: 54, done.
remote: Counting objects: 100% (54/54), done.
remote: Compressing objects: 100% (49/49), done.
remote: Total 54 (delta 4), reused 49 (delta 1), pack-reused 0 (from 0)
Receiving objects: 100% (54/54), 12.73 MiB | 11.95 MiB/s, done.
Resolving deltas: 100% (4/4), done.


In [2]:
!pip install --quiet google-cloud-bigquery

# Import libraries
from google.cloud import bigquery
from google.colab import auth

# Authenticate your Google account
auth.authenticate_user()

# Set up BigQuery client (use your GCP project ID)
project_id = 'akashapiplayground'  # Replace with your actual project ID
client = bigquery.Client(project=project_id)

In [22]:
# ICD9. 430. Subarachnoid hemorrage
# ICD9. 5849. Acute kidney failure NOS
# Filter those diagnoses where billing was for 430
query = """

  SELECT *
  FROM physionet-data.mimiciii_clinical.diagnoses_icd
  WHERE icd9_code  IN ('430', '5849')
  """

query_job = client.query(query)
diagnoses = query_job.to_dataframe()
len(diagnoses)

9777

In [25]:
arr_subject_id=[]
arr_hadm_id=[]
for row in range(0, len(diagnoses)):
  if(diagnoses.loc[row, 'ICD9_CODE'] in ['5849', '430']):
    arr_subject_id.append(diagnoses.loc[row, 'SUBJECT_ID'])
    arr_hadm_id.append(diagnoses.loc[row, 'HADM_ID'])

print(f"{len(arr_subject_id)=}")

len(arr_subject_id)=9777


In [9]:
# Filter those notevents where subejct_id and hadm_id matches the one returned
# from diagnoses_icd query
query = """

  SELECT *
  FROM physionet-data.mimiciii_notes.noteevents
  """

query_job = client.query(query)
noteevents = query_job.to_dataframe()
len(noteevents)

2083180

In [10]:
import pandas as pd

In [26]:
df_noteevents_filtered = noteevents[noteevents['SUBJECT_ID'].isin(arr_subject_id) & noteevents['HADM_ID'].isin(arr_hadm_id)]
len(df_noteevents_filtered)

374555

In [28]:
print(df_noteevents_filtered['CATEGORY'].unique())
print(df_noteevents_filtered.iloc[0]['TEXT'])

['Nursing' 'Radiology' 'Nursing/other' 'ECG' 'Nutrition' 'Physician '
 'General' 'Discharge summary' 'Rehab Services' 'Respiratory ' 'Echo'
 'Case Management ' 'Social Work' 'Consult' 'Pharmacy']
68 yo F with extensive PMH: MI x 3; 70 % occluded LAD even with stent
   in '[**36**]; CABG '[**37**]; arthritis; ulcers; gastritis; reflux;
   diver6ticulitis; CRI/RAS-s/p stent '[**40**]; Left carotid stent '[**36**]; urinary
   incontinence; endometrial polyps; osteoarthritis; polymalagia;
   rheumatoid; PVD; stenting of distal aorta and iliac arteries '[**40**];
   hypercholesterol; Newly dx Squamous cell Lung CA [**5-29**] s/p LLL resection
   w/ New onset Afib after lung resection on Amiodarone, Metoprolol and
   Coumadin .   On [**6-23**] presented to ED w/ BRBPR x3 was hemodynamically
   stable adm to floor.   On [**6-24**]  active bleed, hypotensive SBP 70
   transferred MICU 7 multiple episodes BRBPR  fluid resusitaion 2Liters
   and subsequent 8UPRBC for falling HCT. [**6-25**] Colo

In [30]:
df_discharge_notes = df_noteevents_filtered[df_noteevents_filtered['CATEGORY'] == 'Discharge summary']
len(df_discharge_notes)

10603

In [31]:
df_discharge_notes.to_csv('df_discharge_notes_430_5849.csv', index=False)